In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

In [4]:
### load model
model_nn = tf.keras.models.load_model('nn_final.h5', compile=False)
model_nn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), 
                loss=tf.keras.losses.binary_crossentropy, 
                metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')])

In [5]:
### load feature transformer
scaler = pickle.load(open('scaler.pkl', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'Models/scaler.pkl'

In [5]:
def nn_predict(x):
    '''
    Takes an input, x, and uses the neural network to predict if the hotel
    reservation is cancelled.
    
    ------------
    Input
    ------------
    The input must be an (n x 16) numpy array or an (n x 16) subset of a named DataFrame.

    If the input is a numpy array, then order of the variables must be:

    'type_of_meal_plan', 'required_car_parking_space', 'room_type_reserved', 'market_segment_type', 'no_of_adults', 'no_of_children', 
    'no_of_weekend_nights', 'no_of_week_nights', 'lead_time', 'repeated_guest', 'no_of_previous_cancellations', 
    'no_of_previous_bookings_not_canceled', 'avg_price_per_room', 'no_of_special_requests', 'arrival_month', 'arrival_date'

    ------------
    Output
    ------------
    The output is either 0 or 1.
    0: 'Cancelled'
    1: 'Not_Cancelled'
    
    '''
    ### transform input
    x_scaled = scaler.transform(x)

    ### make prediction
    threshold = 0.5
    pred = model_nn.predict(x_scaled)
    pred = np.where(pred >= threshold, 1, 0)

    return pred

### Example Use

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score

In [7]:
### load in data set and drop 'Booking_ID'
hotel_df = pd.read_csv('Hotel Reservations.csv')
hotel_df.drop(['Booking_ID'], inplace=True, axis=1)

In [8]:
### X will not include 'booking_status' or 'arrival_year'
X = hotel_df.drop(['booking_status', 'arrival_year'], axis=1)

### y must be binary
y = hotel_df['booking_status']
y = LabelBinarizer().fit_transform(y)

In [12]:
### train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=643)

In [13]:
### determine predicted values
train_pred = nn_predict(X_train)
test_pred = nn_predict(X_test)

227/227 [==============================] - 0s 1ms/step


In [14]:
### accuracy
print(f"Training Accuracy:  {100*accuracy_score(y_train, train_pred):.2f}%")
print(f"Testing Accuracy:  {100*accuracy_score(y_test, test_pred):.2f}%")

Training Accuracy:  88.36%
Testing Accuracy:  88.05%
